In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet
import bs4 as bs
import warnings
import urllib.request
import nltk
import random
import string
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

In [18]:
from nltk.tokenize import *;
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
# Filter warnings
warnings.filterwarnings('ignore')

# Greetings and synonyms
greeting_inputs = ['hey', 'whats up', 'good morning', 'good evening', 'morning', 'evening', 'hello there', 'hey there']
synonyms = [lemma.name() for syn in wordnet.synsets('hello') for lemma in syn.lemmas()]
greeting_inputs += synonyms
print(greeting_inputs)
# Greeting responses
greeting_responses = [
    'Hello! How can I help you?',
    'Hey there! What can I assist you with?',
    'Hi! Curious about Jenkins? Ask away!',
    'Greetings! How may I be of service?',
    'Hi there! Ready to explore Jenkins?',
    'Hello, friend! Jenkins-related questions, perhaps?',
    'Hiya! What do you need help with today?',
    'Hello! Looking for information about Jenkins?',
    'Hey! Excited to answer your Jenkins queries!',
    'Hi! Feel free to ask anything about Jenkins.',
    'Greetings and salutations! Jenkins expertise at your service!',
    'Hello, my friend! Need insights about Jenkins?',
    'Hey, hey! Questions about Jenkins? Fire away!',
    'Hello there! Ready to dive into the world of Jenkins?',
    'Hi! Let\'s tackle your Jenkins-related questions!',
    'Heya! Inquisitive about Jenkins? I\'m here to help!',
    'Hello! Eager to assist you with your Jenkins inquiries!',
    'Hey! Curious about Jenkins? I\'m all ears!',
    'Hi! Jenkins knowledge at your disposal!',
    'Hello, my curious friend! Jenkins knowledge awaits!',
    'Hey there! I\'m here to help you explore Jenkins!',
    'Hi! What Jenkins topics are you interested in?',
    'Hello! Let\'s unravel the mysteries of Jenkins!',
    'Hey! I\'m your go-to for all things Jenkins!',
    'Hi! Ready to dive into Jenkins-related discussions?',
    'Hello! Jenkins insights just a question away!',
    'Greetings! Exploring Jenkins? I\'m here to guide you!',
    'Hey there! How can I assist your Jenkins journey?',
    'Hello! Jenkins knowledge is my specialty!',
    'Hey! Curious about Jenkins? I have the answers!',
    'Hi! Ready to delve into the realm of Jenkins?',
    'Hello! Let\'s uncover the wonders of Jenkins!',
    'Hey there! I\'m here to make Jenkins understandable!',
    'Hiya! Seeking Jenkins enlightenment? I\'m here!',
    'Hello! Jenkins insights just one question away!',
    'Hi! Curious about Jenkins? I\'ve got the scoop!',
    'Hello! Ready to explore Jenkins\'s intricacies?',
    'Hey! Jenkins queries? I\'m the answer supplier!',
    'Hiya! In need of Jenkins knowledge? I\'m here!',
    'Hello! Jenkins enlightenment is my mission!',
    'Hey! Seeking Jenkins expertise? Look no further!',
    'Hi! Jenkins discussions? Count me in!',
    'Hello! Curious about Jenkins? Let\'s chat!',
    'Hey there! I\'m all ears for your Jenkins inquiries!',
    'Hi! Ready to dive into the world of Jenkins?'
]
# Conversation inputs and responses
conversation_pairs = {
    'how are you': ['Great! How about you?', 'I am just a bot, but I\'m here to help!'],
    'what are you doing': ['Answering questions about Jenkins!', 'Chatting with you!', 'Helping users like you!'],
    'who made you': ['Aamna Zahid made me.', 'I was created by Aamna Zahid.', 'I owe my existence to Aamna Zahid.'],
    'who are you': ['I am called Jenkins Bot made by Aamna Zahid', 'I\'m a Jenkins Bot created by Aamna.', 'I\'m the Jenkins Bot developed by Aamna Zahid.'],
    'how can you help': ['I can provide information about Jenkins, answer your queries, and more!', 'I can assist with Jenkins-related questions and more.', 'I\'m here to offer help and information about Jenkins.'],
    'thank you': ['You\'re welcome! If you have more questions, feel free to ask.', 'No problem! If you need further assistance, let me know.', 'Glad I could help! Feel free to reach out anytime.'],
}

# Responses to questions
jenkins_question_answers = {
    'what is Jenkins': 'Jenkins is an open-source automation server used for continuous integration and continuous delivery of software.',
    'how to install Jenkins': 'You can install Jenkins by downloading the WAR file and running it using Java, or you can use package managers like apt or yum.',
    'what is a Jenkins pipeline': 'A Jenkins pipeline is a set of code that defines your continuous integration/continuous delivery (CI/CD) process as code.',
    'how to create a Jenkins pipeline': 'You can create a Jenkins pipeline using a Jenkinsfile, which defines the pipeline stages, steps, and configurations.',
    'what is Jenkinsfile': 'A Jenkinsfile is a text file that defines the stages, steps, and configuration of a Jenkins pipeline.',
    'what is a Jenkins agent': 'A Jenkins agent (also known as a slave) is a machine that executes the build processes as instructed by the Jenkins master.',
    'how to configure Jenkins agent': 'You can configure a Jenkins agent by adding it to the Jenkins master, specifying its labels, and configuring its settings.',
    'what is Jenkins Job DSL': 'Jenkins Job DSL is a plugin that allows you to define jobs and pipelines programmatically using a Groovy-based domain-specific language.',
    'how to trigger a Jenkins job': 'Jenkins jobs can be triggered manually by users or automatically based on events such as code commits or timer triggers.',
    'what is Jenkinsfile pipeline syntax': 'Jenkinsfile pipeline syntax is a domain-specific language used to define the structure and behavior of pipelines in Jenkins.',
    'how to integrate Jenkins with Git': 'You can integrate Jenkins with Git repositories by configuring source code management settings in your pipeline.',
    'what is Jenkins Blue Ocean': 'Jenkins Blue Ocean is a modern, user-friendly interface for Jenkins pipelines that provides a visual representation of your CI/CD process.',
    'how to schedule a Jenkins job': 'You can schedule Jenkins jobs using the built-in cron-like syntax in the pipeline or by using the Schedule Build plugin.',
    'what is Jenkins plugin': 'Jenkins plugins are add-ons that extend the functionality of Jenkins by providing additional features and integrations.',
    'how to install Jenkins plugins': 'You can install Jenkins plugins through the Jenkins web interface by navigating to the "Manage Jenkins" > "Manage Plugins" section.',
    'what is Jenkins declarative pipeline': 'Jenkins declarative pipeline is a simplified and structured way to define pipelines using a more human-readable syntax.',
    'how to archive artifacts in Jenkins': 'You can archive artifacts in Jenkins by using the "Archive the artifacts" post-build action in your pipeline.',
    'what is Jenkins shared library': 'Jenkins shared library allows you to define reusable pipeline code that can be shared across multiple projects.',
    'how to secure Jenkins server': 'You can secure the Jenkins server by enabling authentication, configuring authorization, and using security plugins.',
    'what is Jenkins X': 'Jenkins X is an open-source project that provides automated CI/CD for cloud-native applications using Kubernetes.',
    'how to run Jenkins on Docker': 'You can run Jenkins on Docker by pulling the official Jenkins image and running a container with the necessary configuration.',
    'what is Jenkins master-slave architecture': 'Jenkins master-slave architecture involves a Jenkins master server that distributes build tasks to multiple slave agents for execution.',
    'how to trigger Jenkins pipeline on code commit': 'You can trigger Jenkins pipelines on code commits by configuring webhooks or using a version control system integration.',
    'what is Jenkins Groovy DSL': 'Jenkins Groovy DSL allows you to define job configurations using Groovy scripts instead of using the Jenkins web interface.',
    'how to backup and restore Jenkins': 'You can backup and restore Jenkins by copying the necessary configuration files, plugins, and data directories.',
    'what is Jenkins environment variables': 'Jenkins environment variables are predefined variables that provide information about the build environment and pipeline execution.',
    'how to run Selenium tests in Jenkins': 'You can run Selenium tests in Jenkins by configuring the necessary build steps and executing test scripts in the pipeline.',
    'what is Jenkins trigger build remotely': 'Jenkins trigger build remotely feature allows you to start a build job by sending an HTTP POST request.',
    'how to parallelize Jenkins pipeline stages': 'You can parallelize Jenkins pipeline stages using the "parallel" directive to execute multiple stages concurrently.',
    'what is Jenkins pipeline syntax generator': 'Jenkins pipeline syntax generator is a tool that helps you generate syntax for different pipeline steps and configurations.',
    'how to set up email notifications in Jenkins': 'You can set up email notifications in Jenkins by configuring the "Editable Email Notification" post-build action.',
    'what is Jenkins scripted pipeline': 'Jenkins scripted pipeline is a way to define pipelines using Groovy scripts, providing more flexibility and customization.',
    'how to trigger downstream jobs in Jenkins': 'You can trigger downstream jobs in Jenkins by using the "Build other projects" post-build action or using the Build Pipeline plugin.',
    'what is Jenkins pipeline triggers': 'Jenkins pipeline triggers determine when a pipeline should start, such as on code commits, timer triggers, or manual triggers.',
    'how to pass parameters in Jenkins pipeline': 'You can pass parameters in Jenkins pipelines by defining parameters in the pipeline configuration and using them in your script.',
    'what is Jenkins DSL seed job': 'Jenkins DSL seed job is a job that generates other jobs using the Job DSL plugin based on predefined scripts.',
    'how to archive Jenkins build artifacts to Nexus': 'You can archive Jenkins build artifacts to Nexus by using the Nexus Artifact Uploader plugin in your pipeline.',
    'what is Jenkinsfile syntax checker': 'Jenkinsfile syntax checker helps you validate the syntax and structure of your Jenkinsfile to avoid pipeline errors.',
    'how to set up Jenkins on AWS': 'You can set up Jenkins on AWS by launching an EC2 instance, installing Jenkins, and configuring security groups and networking.',
    'what is Jenkins agent-to-master security subsystem': 'Jenkins agent-to-master security subsystem ensures secure communication between Jenkins agents and the master server.',
    'how to trigger Jenkins pipeline from another pipeline': 'You can trigger Jenkins pipelines from another pipeline using the "build" step or the "build" step of the Jenkins DSL.',
    'what is Jenkins performance plugin': 'Jenkins performance plugin allows you to monitor and analyze the performance of your builds and pipelines.',
    'how to set up Jenkins on Kubernetes': 'You can set up Jenkins on Kubernetes by deploying Jenkins as a containerized application using Kubernetes resources.',
    'what is Jenkins parallel stages': 'Jenkins parallel stages allow you to run multiple stages concurrently, optimizing the execution time of your pipeline.',
    'how to set up Jenkins on Ubuntu': 'You can set up Jenkins on Ubuntu by adding the Jenkins repository, installing the package, and configuring the service.',
    'what is Jenkins integration with Jira': 'Jenkins integration with Jira allows you to automatically link builds and deployments to Jira issues for better tracking.',
    'how to create a Jenkins job using Job DSL': 'You can create a Jenkins job using Job DSL by defining the job configuration in a Groovy script and running it as a seed job.',
    'what is Jenkins job build history': 'Jenkins job build history provides a record of all the builds and their outcomes for a particular job over time.',
    'how to run a Jenkins pipeline on a schedule': 'You can run a Jenkins pipeline on a schedule using the cron syntax in the pipeline configuration or using the Schedule Build plugin.',
    'what is Jenkins Blue Ocean pipeline editor': 'Jenkins Blue Ocean pipeline editor is a graphical interface that allows you to design and edit Jenkins pipelines visually.',
    'how to trigger a Jenkins pipeline manually': 'You can trigger a Jenkins pipeline manually by clicking the "Build Now" button on the Jenkins web interface for the pipeline job.',
    'what is Jenkins distributed build': 'Jenkins distributed build refers to the practice of distributing build tasks to multiple agents for faster and more efficient execution.',
    'how to use Jenkins DSL scripts': 'You can use Jenkins DSL scripts to define job configurations and pipeline structures using Groovy scripts in the Job DSL plugin.',
    'what is Jenkins pipeline promotion': 'Jenkins pipeline promotion allows you to promote a build to a specified environment or stage after successful execution.',
    'how to set up Jenkins on CentOS': 'You can set up Jenkins on CentOS by adding the Jenkins repository, installing the package, and starting the Jenkins service.',
    'what is Jenkins build pipeline plugin': 'Jenkins build pipeline plugin provides a visual representation of your CI/CD process by displaying connected jobs and their relationships.',
    'how to run Jenkins in a Docker container': 'You can run Jenkins in a Docker container by pulling the official Jenkins image and running a container with the necessary settings.',
    'what is Jenkins LTS': 'Jenkins LTS (Long-Term Support) is a release branch that provides stability and security updates over an extended period of time.',
    'how to trigger a Jenkins job remotely': 'You can trigger a Jenkins job remotely by using the "Build Now" remote trigger or by using plugins like the Remote Trigger plugin.',
    'what is Jenkins Git plugin': 'Jenkins Git plugin allows you to integrate Jenkins with Git repositories for automated builds triggered by code commits.',
}

lemmatizer = WordNetLemmatizer()

# Pre-process tokens
def preprocess_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens if token not in string.punctuation]

# Processed conversation pairs
processed_conversation_pairs = {key: preprocess_tokens(value) for key, value in conversation_pairs.items()}
print(processed_conversation_pairs)
# Generate greeting response
def generate_greeting_response(user_input):
    if user_input in greeting_inputs:
        return random.choice(greeting_responses)

# Generate conversation response
def generate_conversation_response(user_input):
  return random.choice(conversation_pairs[user_input])
   
def generate_answer_response(user_input):
    user_tokens = preprocess_tokens(word_tokenize(user_input))
    for key, value in jenkins_question_answers.items():
        if all(token in preprocess_tokens(word_tokenize(key)) for token in user_tokens):
            return value


print('Hi! I am Jenkbot. Feel free to ask me anything about Jenkins:')
while True:
    user_input = input().lower()
    if user_input == 'bye':
        print('Jenkbot: Goodbye! Stay safe and curious!')
        break

    greeting_response = generate_greeting_response(user_input)
    if greeting_response:
        print('Jenkbot:', greeting_response)
        continue

    conversation_response = generate_conversation_response(user_input)
    if conversation_response:
        print('Jenkbot:', conversation_response)
        continue

    answer_response = generate_answer_response(user_input)
    if answer_response:
        print('Jenkbot:', answer_response)
        continue



['hey', 'whats up', 'good morning', 'good evening', 'morning', 'evening', 'hello there', 'hey there', 'hello', 'hullo', 'hi', 'howdy', 'how-do-you-do']
{'how are you': ['Great! How about you?', "I am just a bot, but I'm here to help!"], 'what are you doing': ['Answering questions about Jenkins!', 'Chatting with you!', 'Helping users like you!'], 'who made you': ['Aamna Zahid made me.', 'I was created by Aamna Zahid.', 'I owe my existence to Aamna Zahid.'], 'who are you': ['I am called Jenkins Bot made by Aamna Zahid', "I'm a Jenkins Bot created by Aamna.", "I'm the Jenkins Bot developed by Aamna Zahid."], 'how can you help': ['I can provide information about Jenkins, answer your queries, and more!', 'I can assist with Jenkins-related questions and more.', "I'm here to offer help and information about Jenkins."], 'thank you': ["You're welcome! If you have more questions, feel free to ask.", 'No problem! If you need further assistance, let me know.', 'Glad I could help! Feel free to reac

KeyboardInterrupt: ignored